In [1]:
import torch
import torch.nn as nn
from sinabs.backend.dynapcnn import DynapcnnNetworkGraph
from sinabs.layers import Merge, IAFSqueeze, SumPool2d
import sinabs.layers as sl
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential

In [2]:
torch.manual_seed(0)

In [3]:
channels = 2
height = 34
width = 34

input_shape = (channels, height, width)

## Network Module

We need to define a `nn.Module` implementing the network we want the chip to reproduce.

In [67]:
class SNN(nn.Module):
    def __init__(self, nb_classes, batch_size, surrogate_fn, min_v_mem=-0.313, spk_thr=2.0) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(2, 1, 2, 1, bias=False)
        self.iaf1 = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)
        self.pool1 = sl.SumPool2d(2,2)

        self.conv2 = nn.Conv2d(1, 8, 2, 1, bias=False)
        self.iaf2 = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)
        self.pool2 = sl.SumPool2d(2,2)

        self.conv3 = nn.Conv2d(8, 16, 2, 1, bias=False)
        self.iaf3 = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)
        self.pool3 = sl.SumPool2d(2,2)

        self.conv4 = nn.Conv2d(16, 32, 2, 1, bias=False)
        self.iaf4 = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

        self.flat = nn.Flatten()

        self.fc1 = nn.Linear(128, 1024, bias=False)
        self.iaf1_fc = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

        self.fc2 = nn.Linear(1024, 512, bias=False)
        self.iaf2_fc = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

        self.fc3 = nn.Linear(512, 256, bias=False)
        self.iaf3_fc = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

        self.fc4 = nn.Linear(256, 128, bias=False)
        self.iaf4_fc = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

        self.fc5 = nn.Linear(128, nb_classes, bias=False)
        self.iaf5_fc = IAFSqueeze(batch_size=batch_size, min_v_mem=min_v_mem, surrogate_grad_fn=surrogate_fn, spike_threshold=spk_thr)

    def forward(self, x):
        # conv 1
        con1_out = self.conv1(x)
        iaf1_out = self.iaf1(con1_out)
        pool1_out = self.pool1(iaf1_out)

        # conv 2
        conv2_out = self.conv2(pool1_out)
        iaf2_out = self.iaf2(conv2_out)
        pool2_out = self.pool2(iaf2_out)

        # conv 3
        conv3_out = self.conv3(pool2_out)
        iaf3_out = self.iaf3(conv3_out)
        pool3_out = self.pool3(iaf3_out)

        # conv 4
        conv4_out = self.conv4(pool3_out)
        iaf4_out = self.iaf4(conv4_out)

        flat_out = self.flat(iaf4_out)
        
        # fc 1
        print(flat_out.shape)
        fc1_out = self.fc1(flat_out)
        iaf1_fc_out = self.iaf1_fc(fc1_out)

        # fc 2
        fc2_out = self.fc2(iaf1_fc_out)
        iaf2_fc_out = self.iaf2_fc(fc2_out)

        # fc 3
        fc3_out = self.fc3(iaf2_fc_out)
        iaf3_fc_out = self.iaf3_fc(fc3_out)

        # fc 4
        fc4_out = self.fc4(iaf3_fc_out)
        iaf4_fc_out = self.iaf4_fc(fc4_out)

        # fc 5
        fc5_out = self.fc5(iaf4_fc_out)
        iaf5_fc_out = self.iaf5_fc(fc5_out)

        return iaf5_fc_out
    
snn = SNN(11, 8, PeriodicExponential())

x = torch.randn((8, *input_shape))

snn(x)

torch.Size([8, 128])


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<ViewBackward0>)

The following exemplifies how each of the layers in the SNN should be grouped together to form a `DynapcnnLayer` instance. Let's check the shapes of the tensors each of the (original) layers in the model inputs and outputs by feeding a fake input through the model:

In [30]:
x = torch.randn((8, *input_shape))

snn(x)

torch.Size([8, 8, 17, 17])
torch.Size([8, 8, 16, 16])
torch.Size([8, 8, 1, 1])
torch.Size([8, 8])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x8 and 800x11)

## DynapcnnNetwork Class

In the constructor of `DynapcnnNetworkGraph` the SNN passed as argument (defined as a `nn.Module`) will be parsed such that each layer is represented in a computational graph (using `nirtorch.extract_torch_graph`). 

The layers are the `nodes` of the graph, while their connectivity (how the outputs from a layer are sent to other layers) is represented as `edges`, represented in a `list` of `tuples`.

Once the constructor finishes its initialization, the `hw_model.dynapcnn_layers` property is a dictionary where each entry represents the ID of a `DynapcnnLayer` instance (an `int` from `0` to `L`), with this entry containing a `DynapcnnLayer` instance where a subset of the layers in the original SNN has been incorporated into, the core such instance has been assigned to, and the list of `DynapcnnLayer` instances (their IDs) the layer targets.

In [7]:
hw_model = DynapcnnNetworkGraph(
    snn,
    discretize=True,
    input_shape=input_shape
)

The `hw_model.to()` call will figure out into which core eac `DynapcnnLayer` instance will be assigned to. Once this assingment is made the instance itself is used to configure the `CNNLayerConfig` instance representing the core's configuration.

If the cores' configuration is valid, each `DynapcnnLayer` instance and their respective destinations will be used to create a computational graph that encodes how the `forward` method of `hw_model.network` (a `nn.Module` using the `DynapcnnLayer` instances) propagates that through the network.

In [8]:
hw_model.to(device="speck2fmodule:0")

Network is valid


The layers comprising our `hw_model` and their respective metadata can be inspected by calling `print` on a `DynapcnnNetworkGraph` instance.

In [9]:
print(hw_model)

---- DynapcnnLayer 0 ----------------------------------------------------------
> layer modules: 
(node 0): Conv2d(2, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
(node 1): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-32768.), batch_size=1, num_timesteps=-1)
(node 2): SumPool2d(norm_type=1, kernel_size=3, stride=3, ceil_mode=False)
(node 3): SumPool2d(norm_type=1, kernel_size=4, stride=4, ceil_mode=False)
> layer destinations: [1, 2]
> assigned core: 0

---- DynapcnnLayer 1 ----------------------------------------------------------
> layer modules: 
(node 4): Conv2d(10, 10, kernel_size=(4, 4), stride=(1, 1), bias=False)
(node 6): IAFSqueeze(spike_threshold=Parameter containing:
tensor(1.), min_v_mem=Parameter containing:
tensor(-32768.), batch_size=1, num_timesteps=-1)
> layer destinations: [2]
> assigned core: 1

---- DynapcnnLayer 2 ----------------------------------------------------------
> layer modules: 
(node 7): Conv